# Waffle House Formal Authentication Tokenizer Concept


In [1]:
# Import libraries

import pandas as pd
from huggingface_hub import notebook_login
from pprint import pprint
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Login to hugging face
notebook_login()

In [3]:
# Import tokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\Ryan Levey\OneDrive\Bureaublad\vandy_cs\CS 6315\final_project\waffle_house_project\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ryan Levey\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

# Waffle House Formal Authentication

In [97]:
# Define Waffle House markers and expected results (expanded with all relevant tokens)
waffle_tokens = {
    "jelly_bottom": "scrambled_eggs",
    "hashbrowns_top": "hashbrowns_selected",
    "tomato_top": "tomato_selected",
    "napkin_top": "oatmeal_selected",
    "jelly_flipped": "wheat_toast",
    "apple_jelly_bottom": "raisin_toast",
    "extra_jelly": "extra_toast",
    "butter_underneath": "dry_toast",
    "mustard_under_jelly": "three_scrambled_eggs",
    "two_jelly_stack": "four_scrambled_eggs"
}

# Tokenize each marker phrase
tokenized_markers = {k: tokenizer(v)["input_ids"] for k, v in waffle_tokens.items()}


from IPython.display import display
display(tokenized_markers)


{'jelly_bottom': [101, 13988, 168, 6471, 102],
 'hashbrowns_top': [101, 1144, 1324, 12725, 6540, 1116, 168, 2700, 102],
 'tomato_top': [101, 26422, 168, 2700, 102],
 'napkin_top': [101, 184, 2980, 3263, 1348, 168, 2700, 102],
 'jelly_flipped': [101, 11773, 168, 17458, 102],
 'apple_jelly_bottom': [101, 187, 15837, 1394, 168, 17458, 102],
 'extra_jelly': [101, 3908, 168, 17458, 102],
 'butter_underneath': [101, 3712, 168, 17458, 102],
 'mustard_under_jelly': [101, 1210, 168, 13988, 168, 6471, 102],
 'two_jelly_stack': [101, 1300, 168, 13988, 168, 6471, 102]}

In [98]:
# Function to authenticate tokens
def authenticate_user(tokens):
    for token, expected_token in tokenized_markers.items():
        if tokens.get(token) != expected_token:
            return False
    return True

In [99]:
# Sample user tokens to authenticate
user_tokens = {
    "jelly_bottom": tokenizer("scrambled_eggs")["input_ids"],
    "hashbrowns_top": tokenizer("hashbrowns_selected")["input_ids"]
}
auth_status = authenticate_user(user_tokens)
print("Access Granted" if auth_status else "Access Denied")


### Verification Functions Based on SMV Rules

In [100]:
# Rule validation functions
def check_scrambled_eggs_rule(tokens):
    return tokens.get("jelly_bottom") == tokenized_markers["jelly_bottom"]

def check_grits_rule(tokens):
    return (tokens.get("hashbrowns_top") == tokenized_markers["hashbrowns_top"] and "grits" not in tokens)

def check_oatmeal_rule(tokens):
    return tokens.get("napkin_top") == tokenized_markers["napkin_top"]

def check_conflict_rule(tokens):
    return not (tokens.get("hashbrowns_top") and tokens.get("tomato_top"))


### Order Validation Function

In [101]:
def validate_order(tokens):
    
    rules = [
        check_scrambled_eggs_rule(tokens),
        check_grits_rule(tokens),
        check_oatmeal_rule(tokens),
        check_conflict_rule(tokens)
    ]
    return all(rules)



In [102]:
# Validate sample order
if validate_order(user_tokens):
    print("Order Valid - Configuration matches SMV rules")
else:
    print("Order Invalid - Conflict or Rule Violation Detected")

### Export to SMV Format for nuXmv Verification

In [103]:
# Export function to create SMV file
def export_to_smv(tokens, filename="generated_wafflehouse.smv"):
    with open(filename, 'w') as smv_file:
        smv_file.write("MODULE main\nVAR\n")

        # List of all relevant variables
        all_vars = list(waffle_tokens.keys()) + ["scrambled_eggs", "grits", "oatmeal", "brown_sugar_top"]
        for var in all_vars:
            smv_file.write(f"    {var} : boolean;\n")

        # INIT section: all variables false
        smv_file.write("\nINIT\n    ")
        init_conditions = " & ".join([f"!{var}" for var in all_vars])
        smv_file.write(init_conditions + ";\n\n")

        # TRANS section based on rules
        smv_file.write("TRANS\n")
        trans_conditions = [
            "(jelly_bottom -> scrambled_eggs)",
            "(hashbrowns_top -> (scrambled_eggs & !grits))",
            "(oatmeal -> (napkin_top & brown_sugar_top))"
        ]
        smv_file.write("    " + " &\n    ".join(trans_conditions) + ";\n\n")

        # LTL specifications
        smv_file.write("-- Formal Specifications\n")
        smv_file.write("LTLSPEC G (jelly_bottom -> scrambled_eggs);\n")
        smv_file.write("LTLSPEC G (oatmeal -> F(napkin_top & brown_sugar_top));\n")
        smv_file.write("LTLSPEC !(hashbrowns_top & tomato_top);\n")

    print(f"SMV model exported to {filename}")

In [104]:
# Export the tokens for formal verification
export_to_smv(user_tokens)